In [48]:
from urllib import request
from bs4 import BeautifulSoup
import csv

In [ ]:
# главная ф-я - проходит по каждой странице (на каждой странице ищет 18 ссылок на статьи, парсит дату, название статьи, кто опубликовал)
def main_parser():
    last_date = "2017-06-08" # последняя дата - критерий остановки парсинга
    page_num = 0 # начальный номер страницы
    flag = True # флаг равен True, пока дата не последняя
    with open('articles_2017_2018_June.csv', 'w+', encoding='utf-8', newline='') as csv_file: # создаем/открываем .csv файл для записи
        while flag: # пока дата не последняя
            page_num+=1 # увеличиваем номер страницы
            url = "https://www.kyivpost.com/page/"+str(page_num)+"/?s&time=all&section=all&type=all&exclusive=no&authors=all" # втсавляем номер страницы в ссылку
            req = request.Request(url, headers={'User-Agent' : "Magic Browser"})  # параметр для http запроса, который говорит что ты не робот
            html = request.urlopen(req).read().decode('utf8') # скачиваем html код страницы с номером page_num
            soup = BeautifulSoup(html, "lxml") # превращаем полученную строку с html-кодом в обьект BeatifulSoup, чтобы легко было парсить
            article_oject = soup.findAll('div', {'class':"grid-3 post-excerpt post-excerpt-simple"}) # находим те участки html-кода, которые отображают статьи - получаем список таких участков - то есть список всех статей на этой странице по сути
            for obj in article_oject: # проходимся по каждому такому участку и для каждой статьи
                datetime = obj.find('time')['datetime'] # парсим дату
                if datetime.startswith(last_date): # проверяем, не последняя ли дата
                    break # если последняя - парсинг остановится
                article_title = obj.find('h2', {'class': 'title'}).text # парсим название статьи
                article_link = obj.find('a')['href'] # парсим ссылку на статью
                published_by = obj.find('div', {'class':"author mt-bigger"}).text # парсим, кто опубликовал
                if article_link.endswith('html'): # дополнительная проверка во избежание ошибок - потом обьясню
                    article_text = get_article(article_link) # используем еще одну написанную нами ф-ю, чтобы достать текст статьи
                    if article_text != 0: # если функция нашла текст статьи то записываем строку в .csv файл
                        row = [article_title.replace('\n', ''), published_by, datetime[:10],  article_text,  article_link.replace('\n\n', '\n')]
                        wr = csv.writer(csv_file, dialect='excel', delimiter = ";")
                        wr.writerow(row)
                        print (article_link)
            print ("----------", page_num, "--------------")
            print ("----------", datetime[:10], "--------------")
   

In [66]:
# вспомагательная ф-я, чтобы спарсить текст статьи по переданной ссылке
def get_article(url):
    req = request.Request(url, headers={'User-Agent' : "Magic Browser"}) 
    html = request.urlopen(req).read().decode('utf8') # скачиваем html-код страницы со статьей 
    soup = BeautifulSoup(html, "lxml") # превращаем полученную строку с html-кодом в обьект BeatifulSoup, чтобы легко было парсить
    article_obg = soup.find('div', {'class': "content-block"}) # находим ту часть кода, где находится статья
    if article_obg is None: # если такой части не нашли, возвращаем 0
        return 0
    article_parts = article_obg.findAll('p') # находим все адзацы статьи 
    final_article = ""
    for tag in article_parts: # соединяем все абзацы в одну строку и возвращаем как текст статьи
        final_article+=' '+tag.text
    return final_article.replace("Found a spelling error? Let us know – highlight it and press Ctrl + Enter.", '')


In [ ]:
# запускаеш парсер тут
main_parser()